In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
import pickle
import numpy as np
import pytransform3d.transformations as pt
import pytransform3d.trajectories as ptr
from pytransform3d.camera import check_transform, transform, vectors_to_points, _make_camera_frame, _make_camera_top

## Trajectories

In [ ]:
def plot_camera(ax=None, M=None, cam2world=None, virtual_image_distance=1.0,
                sensor_size=(1920, 1080), ax_s=1, strict_check=True,
                **kwargs):  # pragma: no cover
    """Plot camera in world coordinates.

    This function is inspired by Blender's camera visualization. It will
    show the camera center, a virtual image plane, and the top of the virtual
    image plane.

    Parameters
    ----------
    ax : Matplotlib 3d axis, optional (default: None)
        If the axis is None, a new 3d axis will be created.

    M : array-like, shape (3, 3)
        Intrinsic camera matrix that contains the focal lengths on the diagonal
        and the center of the the image in the last column. It does not matter
        whether values are given in meters or pixels as long as the unit is the
        same as for the sensor size.

    cam2world : array-like, shape (4, 4), optional (default: I)
        Transformation matrix of camera in world frame. We assume that the
        position is given in meters.

    virtual_image_distance : float, optional (default: 1)
        Distance from pinhole to virtual image plane that will be displayed.
        We assume that this distance is given in meters. The unit has to be
        consistent with the unit of the position in cam2world.

    sensor_size : array-like, shape (2,), optional (default: [1920, 1080])
        Size of the image sensor: (width, height). It does not matter whether
        values are given in meters or pixels as long as the unit is the same as
        for the sensor size.

    ax_s : float, optional (default: 1)
        Scaling of the new matplotlib 3d axis.

    strict_check : bool, optional (default: True)
        Raise a ValueError if the transformation matrix is not numerically
        close enough to a real transformation matrix. Otherwise we print a
        warning.

    kwargs : dict, optional (default: {})
        Additional arguments for the plotting functions, e.g. alpha.

    Returns
    -------
    ax : Matplotlib 3d axis
        New or old axis

    Raises
    ------
    ValueError
        If input is not valid
    """

    cam2world = check_transform(cam2world, strict_check=strict_check)

    camera_center_in_world = cam2world[:3, 3]
    focal_length = np.mean(np.diag(M[:2, :2]))
    # X points to right, y points down, z points forward
    sensor_corners_in_cam = np.array([
        [focal_length, 0, 0], #Top left
        [focal_length, 0, sensor_size[1]], #Top right
        [focal_length, sensor_size[0], sensor_size[1]], #Bottom right
        [focal_length, sensor_size[0], 0], #Bottom left
    ])
    sensor_corners_in_cam[:, 1] -= M[0, 2]
    sensor_corners_in_cam[:, 2] -= M[1, 2]
    sensor_corners_in_world = transform(
        cam2world, vectors_to_points(sensor_corners_in_cam))[:, :3]
    virtual_image_corners = (
        virtual_image_distance / focal_length *
        (sensor_corners_in_world - camera_center_in_world[np.newaxis]) +
        camera_center_in_world[np.newaxis])

    if "c" in kwargs:
        color = kwargs.pop("c")
    elif "color" in kwargs:
        color = kwargs.pop("color")
    else:
        color = "k"

    _make_camera_frame(
        ax, virtual_image_corners, camera_center_in_world, color, **kwargs)
    _make_camera_top(ax, virtual_image_corners, color, **kwargs)
    ax.scatter(camera_center_in_world[0], camera_center_in_world[1],
               camera_center_in_world[2], color=color, **kwargs)

    return ax

In [ ]:
%matplotlib ipympl
from itertools import cycle

def calc_quat_angle_error_single(label, pred):
    q1 = np.array(pred / np.linalg.norm(pred))
    q2 = np.array(label / np.linalg.norm(label))
    d = np.abs(np.sum(np.multiply(q1,q2))) # Here we have abs()

    d = np.clip(d, a_min=-1, a_max=1)
    error = 2 * np.degrees(np.arccos(d))
    return error

def plot_pred_and_targ(pred_poses, targ_poses, x_lim=[0, -10], y_lim=[-10, 0], z_lim=[-1, -5], start=0, end=50, step=10):
    sensor_size = np.array([0.036, 0.024])
    intrinsic_matrix = np.array([
        [0.05, 0, sensor_size[0] / 2.0],
        [0, 0.05, sensor_size[1] / 2.0],
        [0, 0, 1]
    ])
    virtual_image_distance = 1
    ax = plt.axes(projection= '3d')
    colors = cycle("rgbcm")
    ax = ptr.plot_trajectory(ax, P=targ_poses[start:end:step], s=0.1,show_direction=False, lw=2, n_frames=10, c='k')
    for targ_pose, c in zip(targ_poses[start:end:step], colors):
        x,y,z,qw,qx,qy,qz = targ_pose
        cam2world = pt.transform_from_pq([x,y,z,qw,qx,qy,qz])
        # default parameters of a camera in Blender
        plot_camera(ax, cam2world=cam2world, M=intrinsic_matrix, sensor_size=sensor_size,
            virtual_image_distance=virtual_image_distance, c=c)

    colors = cycle("rgbcm")
    ax = ptr.plot_trajectory(ax, P=pred_poses[start:end:step], s=0.1,show_direction=False, lw=2, n_frames=10, c='y')
    for pred_pose, c in zip(pred_poses[start:end:step], colors):
        x,y,z,qw,qx,qy,qz = pred_pose
        cam2world = pt.transform_from_pq([x,y,z,qw,qx,qy,qz])
        # default parameters of a camera in Blender
        plot_camera(ax, cam2world=cam2world, M=intrinsic_matrix, sensor_size=sensor_size,
            virtual_image_distance=virtual_image_distance, c=c)
    # calculate losses
    t_criterion = lambda t_pred, t_gt: np.linalg.norm(t_pred - t_gt)
    q_criterion = calc_quat_angle_error_single
    t_loss = np.asarray([t_criterion(p, t) for p, t in zip(pred_poses[start:end:step, :3],
                                                           targ_poses[start:end:step, :3])])
    q_loss = np.asarray([q_criterion(p, t) for p, t in zip(pred_poses[start:end:step, 3:],
                                                           targ_poses[start:end:step, 3:])])

    print('Error in translation: median {:3.2f} m,  mean {:3.2f} m\n'
            'Error in rotation: median {:3.2f} degrees, mean {:3.2f} degree'.format(np.median(t_loss), np.mean(t_loss),
                                                                                    np.median(q_loss), np.mean(q_loss)))
    ax.set_xlim(x_lim)
    ax.set_ylim(y_lim)
    ax.set_zlim(z_lim)
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('z (m)')

In [ ]:
with open('./data/AirSim_building_new_mapnet_noise_none_baseline.pkl', 'rb') as f:
    data = pickle.load(f)
targ_poses = data['targ_poses']
pred_poses = data['pred_poses']
plt.figure()
# plot_pred_and_targ(pred_poses, targ_poses, [-2, -8], [-10, -2], [-1, -4], 100, 140, 10)
plot_pred_and_targ(pred_poses, targ_poses)
plt.legend()
plt.show()

In [ ]:
with open('./data/AirSim_building_new_mapnet_noise_none_imu_0_5.pkl', 'rb') as f:
    data = pickle.load(f)
targ_poses = data['targ_poses']
pred_poses = data['pred_poses']
plt.figure()
# plot_pred_and_targ(pred_poses, targ_poses, [-2, -8], [-10, -2], [-1, -4], 100, 140, 10)
plot_pred_and_targ(pred_poses, targ_poses)

In [ ]:
with open('./data/AirSim_building_new_mapnet_noise_v1_baseline.pkl', 'rb') as f:
    data = pickle.load(f)
targ_poses = data['targ_poses']
pred_poses = data['pred_poses']
plt.figure()
# plot_pred_and_targ(pred_poses, targ_poses, [-2, -8], [-10, -2], [-1, -4], 100, 140, 10)
plot_pred_and_targ(pred_poses, targ_poses)
# plt.legend(["Target", "Prediction"])
ax = plt.gca()
leg = ax.legend(["Target", "Prediction"])
leg.legendHandles[0].set_color('k')
leg.legendHandles[1].set_color('y')
plt.title("Baseline trained on Noisy_v1")
plt.show()

In [ ]:
with open('./data/AirSim_building_new_mapnet_noise_v1_imu_0_5.pkl', 'rb') as f:
    data = pickle.load(f)
targ_poses = data['targ_poses']
pred_poses = data['pred_poses']
plt.figure()
# plot_pred_and_targ(pred_poses, targ_poses, [-2, -8], [-10, -2], [-1, -4], 100, 140, 10)
plot_pred_and_targ(pred_poses, targ_poses)
ax = plt.gca()
leg = ax.legend(["Target", "Prediction"])
leg.legendHandles[0].set_color('k')
leg.legendHandles[1].set_color('y')
plt.title("Weighted loss function model (alpha = 0.5) trained on Noisy_v1")
plt.show()